In [1]:
'''
@Author: Haihui Pan
@Date: 2021-12-21
@Ref: https://pandas.pydata.org/docs/user_guide/10min.html
'''
import pandas as pd

## DataFrame常用方法


* 查看数据 df.head(),df.tail()
* 数据大小：df.shape
* 字段的基础统信息：df.describe(percentiles=[.05, .25, .75, .95]) #指定分位数
* 字段取值的统计：df.value_counts()
* 新增列 
* 字段非空信息：df.info()
* 缺失值处理
   * 删除缺失值(删除行，删除列)
   * 缺失值填充
* 遍历DataFrame
* 行列处理函数：df.apply (类别映射，列相加)

### 查看数据

In [2]:
#读取csv数据
df_student=pd.read_csv('student.csv')

#查看前3个数据，默认为5
df_student.head(3) 

,name,age,sex
0,Tom,8,M
1,Mike,7,M
2,Mary,8,F


In [3]:
#查看最后3个数据
df_student.tail(3)

,name,age,sex
3,Jack,9,M
4,Tim,7,M
5,Jim,8,M


### 数据大小

In [5]:
#(6,3)表示6行，每行3个元素
df_student.shape

(6, 3)

### 基础统计信息

In [6]:
# percentiles用于定义需要统计的分位数（只有数值类型的列才会被统计）
df_student.describe(percentiles=[.05, .25, .75, .95])

,age
count,6.000000
mean,7.833333
std,0.752773
min,7.000000
5%,7.000000
25%,7.250000
50%,8.000000
75%,8.000000
95%,8.750000
max,9.000000


### 统计列取值数

In [9]:
#统计每一行的记录是否有重复
df_student.value_counts()

name  age  sex
Jack  9    M      1
Jim   8    M      1
Mary  8    F      1
Mike  7    M      1
Tim   7    M      1
Tom   8    M      1
dtype: int64

In [11]:
#统计[age,sex]两列的重复情况,类似SQL的group by效果 
df_student[['age','sex']].value_counts()

age  sex
7    M      2
8    M      2
     F      1
9    M      1
dtype: int64

In [13]:
#统计单列的取值情况
df_student['sex'].value_counts()

M    5
F    1
Name: sex, dtype: int64

### 新增列

In [14]:
#在现有的df基础上添加一列score
score=[90,88,None,85,99,85]

#设定好列名则可以直接进行赋值
df_student['score']=score

df_student.head()

,name,age,sex,score
0,Tom,8,M,90.0
1,Mike,7,M,88.0
2,Mary,8,F,NaN
3,Jack,9,M,85.0
4,Tim,7,M,99.0


### 查看数据缺失情况

In [15]:
# score列只有5个非空，因此有一行是None
df_student.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    6 non-null      object 
 1   age     6 non-null      int64  
 2   sex     6 non-null      object 
 3   score   5 non-null      float64
dtypes: float64(1), int64(1), object(2)
memory usage: 320.0+ bytes


### 缺失值处理

* 通常收集到的原始数据会包含一些缺失的数据。对于缺失的数据，一般有2种做法，即删除包含缺失值的记录，或者对缺失值进行填充
    * 删除行：当缺失的数据集中在某一行时，则可以丢弃该行
    * 删除列：当缺失的数据集中在某一列时，则可以丢弃该列
    * 缺失值填充：可以采用常数填充，采用所在列均值填充...

### 删除行

In [19]:
#删除行,how='any'表示存在缺失值则删除该行
df_drop=df_student.dropna(axis=0, how='any',inplace=False)

#从(6,4)->(5,4)
df_drop.shape

(5, 4)

In [17]:
# thresh需要几个非空数据，才能留下
df_drop=df_student.dropna(axis=0, how='any',thresh=3,inplace=False)

#thresh=3则不会删除任何数据
df_drop.shape

(6, 4)

In [18]:
# thresh需要几个非空数据，才能留下
df_drop=df_student.dropna(axis=0, how='any',thresh=4,inplace=False)

#thresh=4则会删除缺失行
df_drop.shape

(5, 4)

### 删除列

In [20]:
#删除score所在列
df_drop=df_student.drop(['score'],axis=1,inplace=False)

df_drop.head()

,name,age,sex
0,Tom,8,M
1,Mike,7,M
2,Mary,8,F
3,Jack,9,M
4,Tim,7,M


### 缺失值填充

In [22]:
#使用常数来进行填充,指定列名要填充值
fill_dict={'score':90}

df_fill=df_student.fillna(fill_dict,inplace=False)
df_fill.head()

,name,age,sex,score
0,Tom,8,M,90.0
1,Mike,7,M,88.0
2,Mary,8,F,90.0
3,Jack,9,M,85.0
4,Tim,7,M,99.0


In [23]:
#使用列均值来进行填充（也可以使用中位数，众数）
fill_dict={'score':df_student['score'].mean()} #中位数:df.median(),众数:df.mode() 

df_fill=df_student.fillna(fill_dict,inplace=False)
df_fill.head()

,name,age,sex,score
0,Tom,8,M,90.0
1,Mike,7,M,88.0
2,Mary,8,F,89.4
3,Jack,9,M,85.0
4,Tim,7,M,99.0


### 遍历DataFrame

In [24]:
#遍历数据
for idx,row in df_student.iterrows():
    print(row['name'],row['age'],row['sex'])

Tom 8 M
Mike 7 M
Mary 8 F
Jack 9 M
Tim 7 M
Jim 8 M


### DataFrame.apply函数

* data.apply()允许用户自己设置函数用于对DataFrame的列进行处理，也使用用于新增列

In [25]:
#基于score列，添加一列level
def get_level(s):
    if s>=90:
        return 'A'
    elif s>=80 and s<90:
        return 'B'
    else:
        return 'C'

#对df['score']的每一行对进行get_level处理
df_student['level']=df_student['score'].apply(get_level)
df_student.head()

,name,age,sex,score,level
0,Tom,8,M,90.0,A
1,Mike,7,M,88.0,B
2,Mary,8,F,NaN,C
3,Jack,9,M,85.0,B
4,Tim,7,M,99.0,A


In [26]:
#将sex列中的M映射为0，F映射为1
def map_sex(s):
    if s=="M":
        return 0
    else:
        return 1
    
df_student['sex']=df_student['sex'].apply(map_sex)
df_student.head()

,name,age,sex,score,level
0,Tom,8,0,90.0,A
1,Mike,7,0,88.0,B
2,Mary,8,1,NaN,C
3,Jack,9,0,85.0,B
4,Tim,7,0,99.0,A


### 类别编码(one-hot)

In [29]:
#将leve转换为one-hot
df_level=pd.get_dummies(df_student[['level']])

df_level.head()

,level_A,level_B,level_C
0,1,0,0
1,0,1,0
2,0,0,1
3,0,1,0
4,1,0,0
